In [1]:
import pandas as pd

blog_train = pd.read_csv('local_data/blog_train.csv')
blog_test = pd.read_csv('local_data/blog_test.csv')
mail_train = pd.read_csv('local_data/mail_train.csv')
mail_test = pd.read_csv('local_data/mail_test.csv')
book_train = pd.read_csv('local_data/book_train.csv')
book_test = pd.read_csv('local_data/book_test.csv')

In [2]:
datasets = {'blog_train': blog_train,
            'blog_test': blog_test,
            'mail_train': mail_train,
            'mail_test': mail_test,
            'book_train': book_train,
            'book_test': book_test,
            }

{print(f'{key}: {len(value.id.unique())}')for key, value in datasets.items()}

blog_train: 18247
blog_test: 960
mail_train: 143
mail_test: 7
book_train: 1227
book_test: 64


{None}